In [ ]:
# import matplotlib to show plots inline.
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import os

import astropy.units as u
import matplotlib as mpl
import sys 
from script_utils import get_opsim_areas

In [ ]:
# import maf python modules
CWD = os.getcwd()
sys.path = [CWD+"/../.."]+sys.path
import rubin_sim.maf.db as db
import rubin_sim.maf.metrics as metrics
import rubin_sim.maf.slicers as slicers
import rubin_sim.maf.stackers as stackers
import rubin_sim.maf.plots as plots
import rubin_sim.maf.metricBundles as metricBundles

In [ ]:
from opsimUtils import *

In [ ]:
NSIDE = 64
folder_mafoutput = "NQSO_test_{0:d}".format(NSIDE)
cwd = os.getcwd()
resultDbPath = cwd+'/../../../scratch/MAFOutput/{}'.format(folder_mafoutput)
metricDataPath = cwd+'/../../../scratch/MAFOutput/{}/MetricData/'.format(folder_mafoutput)

In [ ]:
# get a dictionary of resultDb from given directory
resultDbs = getResultsDbs(resultDbPath)

In [ ]:
# retrieve metricBundles for each opsim run and store them in a dictionary
bundleDicts_raw = dict()
for runName in resultDbs:
    bundleDicts_raw[runName] = bundleDictFromDisk(resultDbs[runName], runName, metricDataPath)

In [ ]:
#Rearrange the bundleDicts_raw dictionary so that the keys are always the same as for the first run.
dbRuns = list(bundleDicts_raw.keys())
Keys = list(bundleDicts_raw[dbRuns[0]].keys())
bundleDicts = dict()
for runName in dbRuns:
    bundleDicts[runName] = dict()
    Keys_raw = bundleDicts_raw[runName]
    for Key in Keys:
        for Key_raw in Keys_raw:
            if Key[1]==Key_raw[1]:
                bundleDicts[runName][Key] = bundleDicts_raw[runName][Key_raw]

In [ ]:
# check keys
dbRuns = list(resultDbs.keys())
bd_keys = list(bundleDicts[dbRuns[0]].keys())
print(bd_keys)

### Process the ExgalM5_with_cuts_AGN the same way we did for the Cadence Note to get the number of quasars. 

In [ ]:
from Filter_Curves.lam_eff import lam_eff

In [ ]:
#sys.path.append("../../../AGN_Photoz_LSST_OpSim")
from Nqso_WFD_miniSurveys.Fast_Nqso import Fast_Nqso

In [ ]:
mbright = {"u":14.7,
           "g":15.7,
           "r":15.8,
           "i":15.8,
           "z":15.3,
           "y":13.9
          }
def get_Nqso(bd, run, filter, wfd_mask=None):
    zmin = 0.3
    zmax = np.min([6.7, (lam_eff['LSST'+filter]/(912.*u.AA)-1.0).to(1.).value])
    for run_key in bd[run].keys():
        if run_key[1]=='ExgalM5_with_cuts_AGN':
            Key = run_key
            break
    mask = bd[run][Key].metricValues.mask
    if wfd_mask is not None:
        mask = mask | wfd_mask[run]
    mlim5 = bd[run][Key].metricValues[~mask]
    pix_area = bd[run][Key].slicer.pixArea*u.sr
    qso_counter = Fast_Nqso("LSST"+filter, "Shen20", "A", area=pix_area)
    Nqso = np.sum(qso_counter.Nqso(zmin, zmax, mbright[filter], mlim5))
    Area = len(mlim5) * pix_area.to(u.deg**2).value
    return Nqso, Area

In [ ]:
filters = ["i"]
Nqso = dict()
Area = dict()
dens = dict()
for filter in filters:
    if filter not in Nqso:
        Nqso[filter] = np.zeros(len(dbRuns))
        Area[filter] = np.zeros(len(dbRuns))
        dens[filter] = np.zeros(len(dbRuns))
    for k, run in enumerate(dbRuns):
        Nqso[filter][k], Area[filter][k] = get_Nqso(bundleDicts, run, filter)
        dens[filter][k] = Nqso[filter][k]/Area[filter][k]

In [ ]:
print(Nqso)
print(Area)

### Now compare with the outputs of QSONumberCountsMetric

In [ ]:
Key = (2, 'QSONumberCountsMetric')
for k,run in enumerate(dbRuns):
    mask = bundleDicts[run][Key].metricValues.mask
    Nqso2 = np.sum(bundleDicts[run][Key].metricValues[~mask])
    print(run)
    print("Nqso from QSONumberCountsMetric: {:.1f}".format(Nqso2))
    print("Nqso from ExgalM5_with_cuts_AGN: {:.1f}".format(Nqso['i'][k]))
    print("Difference: {:.1f}".format(Nqso2-Nqso['i'][k]))
    print()